In [2]:
from typing import Any, Optional, Union, List
from pydantic import BaseModel, PositiveInt, PositiveFloat
from datetime import datetime
import os
from sortedcontainers import SortedList
from enum import Enum, auto

In [78]:
class OrderStatus(Enum):
    CREATED = auto()
    PARTIALLY_FILLED = auto()
    FILLED = auto()
    MODIFIED = auto()
    CANCELLED = auto()
    RESTORED = auto()
    EXPIRED = auto()

    @property
    def is_active(self):
        return self in {
            OrderStatus.CREATED,
            OrderStatus.PARTIALLY_FILLED,
            OrderStatus.MODIFIED,
            OrderStatus.RESTORED,
        }


class OrderType(Enum):
    ASK = "ask"
    BID = "bid"


class Order(BaseModel):
    id: PositiveInt
    order_type: OrderType
    price: PositiveFloat
    volume: PositiveInt
    owner_id: PositiveInt
    status: Optional[OrderStatus] = None
    created: Optional[datetime] = None
    updated: Optional[datetime] = None
    listed: Optional[datetime] = None

    def __init__(self, **data):
        # logging here
        super().__init__(**data)
        if self.status is None:
            self.status = OrderStatus.CREATED
        now = datetime.now()
        if self.created is None:
            self.created = now
        if self.updated is None:
            self.updated = now

    def __setattr__(self, name: str, value: Any) -> None:
        if name != "updated":
            super().__setattr__("updated", datetime.now())
        # logging here
        return super().__setattr__(name, value)

    def __lt__(self, other: Union[int, float, "Order"]):
        if isinstance(other, (int, float)):
            return self.price < other
        if isinstance(other, Order):
            return self.price < other.price
        return NotImplemented

    def __gt__(self, other: Union[int, float, "Order"]):
        if isinstance(other, (int, float)):
            return self.price > other
        if isinstance(other, Order):
            return self.price > other.price
        return NotImplemented

    def __le__(self, other: Union[int, float, "Order"]):
        if isinstance(other, (int, float)):
            return self.price <= other
        if isinstance(other, Order):
            return self.price <= other.price
        return NotImplemented

    def __ge__(self, other: Union[int, float, "Order"]):
        if isinstance(other, (int, float)):
            return self.price >= other
        if isinstance(other, Order):
            return self.price >= other.price
        return NotImplemented


class OrderList:
    """
    Order List is a collection of orders. It is used to store and manage orders.
    Active orders are stored in a sorted list with search complexity of O(log n) and insertion complexity of O(n).
    All orders are indexed by their id for O(1) access.
    Active orders can be reached through bisect operations.
    """

    def __init__(
        self,
        order_type: OrderType,
        order_list: Optional[List[Order]] = None,
    ) -> None:
        """
        Initializes a collection of orders of a specified type. All orders within the collection must share the same order type.

        :param order_type: The type of orders to be stored in the list, e.g., 'ask' or 'bid'. The specific types are defined in the OrderType enum.
        :type order_type: OrderType
        :param order_list: Initial list of orders to be added to the collection, defaults to None. Each order in the list is added to the collection using the 'add' method logic.
        :type order_list: Optional[List[Order]], optional
        """
        self.__order_list = SortedList()
        self.__ids = {}
        self.otype = order_type
        if order_list:
            for order in order_list:
                self.add(order)

    def add(self, order: Order, tolist: Union[bool, str] = "auto") -> None:
        """
        Adds an order to the collection. The order is only added to the sorted list if it meets the criteria defined by the 'tolist' parameter. By default ('auto'), active orders are added to the sorted list.

        :param order: The order to be added to the collection.
        :type order: Order
        :param tolist: Specifies how the order should be added to the list. 'auto' adds active orders automatically; True or 'y'/'yes' always adds; False or 'n'/'no' never adds. Defaults to 'auto'.
        :type tolist: Union[bool, str], optional
        :raises ValueError: If the order's type does not match the collection's type.
        :raises ValueError: If 'tolist' is given an invalid value.
        """
        if order.order_type != self.otype:
            raise ValueError("Order type must be the same as the list type")
        if tolist == "auto":
            if order.status.is_active:
                if order.listed is None:
                    order.listed = datetime.now()
                self.__order_list.add(order)
        elif tolist is True or tolist in ["y", "yes"]:
            self.__order_list.add(order)
        elif tolist is False or tolist in ["n", "no"]:
            pass
        else:
            raise ValueError("Invalid value for tolist")
        self.__ids[order.id] = order

    def bisect_left(
        self, order: Union[Order, float], include_right: bool = True
    ) -> List[Order]:
        """
        Performs a bisect left operation on the sorted list to find the position to insert 'order' or 'price'.
        Returns a segment of the list based on the bisect position.

        :param order: Either an Order object or a float price value to perform the bisect operation.
        :type order: Union[Order, float]
        :param include_right: If True, returns the segment of the list from the bisect position to the end (inclusive of the position).
                              If False, returns the segment up to the bisect position (exclusive).
                              Defaults to True.
        :type include_right: bool, optional
        :return: A list of Orders filtered based on the bisect operation.
        :rtype: List[Order]
        """
        cid = self.__order_list.bisect_left(order)
        if include_right:
            return self.__order_list[cid:]
        return self.__order_list[:cid]

    def bisect_right(
        self, order: Union[Order, float], include_left: bool = True
    ) -> List[Order]:
        """
        Performs a bisect right operation on the sorted list to find the position to insert 'order' or 'price'.
        Returns a segment of the list based on the bisect position.

        :param order: Either an Order object or a float price value to perform the bisect operation.
        :type order: Union[Order, float]
        :param include_left: If True, returns the segment of the list up to the bisect position (inclusive).
                             If False, returns the segment of the list from the bisect position to the end (exclusive).
                             Defaults to True.
        :type include_left: bool, optional
        :return: A list of Orders filtered based on the bisect operation.
        :rtype: List[Order]
        """
        cid = self.__order_list.bisect_right(order)
        if include_left:
            return self.__order_list[:cid]
        return self.__order_list[cid:]

    def unlist(self, order: Union[Order, int], order_status: OrderStatus) -> None:
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        if not order.status.is_active:
            raise ValueError("Order is not active.")
        if order_status.is_active:
            raise ValueError("Order cannot be unlisted with active status.")
        self.__order_list.remove(order)
        order.status = order_status

    def relist(self, order: Union[Order, int], order_status: OrderStatus = OrderStatus.RESTORED) -> None:
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        if not order_status.is_active:
            raise ValueError("Active order cannot have non-active status")
        order.status = order_status
        order.listed = datetime.now()
        self.__order_list.add(order)

    def remove(self, order: Union[Order, int]) -> None:
        """
        Remove an order from the collection.

        :param order: order to be removed, it can be either an order or an order id
        :type order: Union[Order, int]
        :raises ValueError: if provided order is invalid type
        :raises ValueError: if order is not found
        """
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")

        if order.status.is_active:
            self.__order_list.remove(order)
        del self.__ids[order.id]

    def expire(self, order: Union[Order, int]) -> None:
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        if not order.status.is_active:
            raise ValueError("Order is not active.")
        self.__order_list.remove(order)
        order.status = OrderStatus.EXPIRED

    def cancel(self, order: Union[Order, int]) -> None:
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        if not order.status.is_active:
            raise ValueError("Order is not active.")
        self.__order_list.remove(order)
        order.status = OrderStatus.CANCELLED

    def fill(self, order: Union[Order, int], volume: int) -> None:
        if volume <= 0:
            raise ValueError("Volume must be positive")
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        if not order.status.is_active:
            raise ValueError("Order is not active.")
        if volume > order.volume:
            raise ValueError("Volume is greater than order volume")
        order.volume -= volume
        if order.volume == 0:
            self.unlist(order, OrderStatus.FILLED)
        else:
            order.status = OrderStatus.PARTIALLY_FILLED

    def modify(
        self,
        order: Union[Order, int],
        price: Optional[float] = None,
        volume: Optional[int] = None,
        relist: bool = True,
    ) -> None:
        if any([price<=0, volume<=0]):
            raise ValueError("Price and volume must be positive")
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        
        if relist:
            if order.status.is_active:
                self.__order_list.remove(order)
            else:
                print("Warning: Order is not active, it will not be relisted")
                relist = False
        order = self.__ids[order.id]
        if price is not None:
            order.price = price
        if volume is not None:
            order.volume = volume
        if relist:
            order.status = OrderStatus.MODIFIED
            self.__order_list.add(order)

    def clear(self) -> None:
        """
        Clears the collection of all orders.
        """
        self.__order_list.clear()
        self.__ids.clear()

    def __getitem__(self, order_id):
        return self.__ids[order_id]

    def __iter__(self):
        return iter(self.__order_list)

    def __len__(self):
        return len(self.__ids)

    def __repr__(self):
        return str(self.__order_list)

# simple tests

In [79]:
import random

### creation

In [80]:
orders = []
for i in range(random.randint(10, 25)):
    orders.append(
        Order(
            id=i+1,
            order_type=OrderType.ASK,
            price=random.uniform(95, 110),
            volume=random.randint(10, 70),
            owner_id=random.randint(1, 10),
        )
    )
orders[:5]

[Order(id=1, order_type=<OrderType.ASK: 'ask'>, price=98.39128386682549, volume=30, owner_id=4, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), listed=None),
 Order(id=2, order_type=<OrderType.ASK: 'ask'>, price=100.66665333771964, volume=30, owner_id=10, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), listed=None),
 Order(id=3, order_type=<OrderType.ASK: 'ask'>, price=100.02901457203792, volume=36, owner_id=6, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), listed=None),
 Order(id=4, order_type=<OrderType.ASK: 'ask'>, price=107.3898071650771, volume=57, owner_id=10, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.d

In [81]:
order_list = OrderList(order_type=OrderType.ASK, order_list=orders)
order_list

SortedList([Order(id=16, order_type=<OrderType.ASK: 'ask'>, price=96.4238155325062, volume=30, owner_id=3, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 188255), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)), Order(id=11, order_type=<OrderType.ASK: 'ask'>, price=96.64354685200499, volume=67, owner_id=3, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)), Order(id=1, order_type=<OrderType.ASK: 'ask'>, price=98.39128386682549, volume=30, owner_id=4, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)), Order(id=3, order_type=<OrderType.ASK: 'ask'>, price=100.029014

### wrong order

In [82]:
wrong_order = Order(
    id=100,
    order_type=OrderType.BID,
    price=100,
    volume=100,
    owner_id=1,
)
order_list.add(wrong_order)
# error

ValueError: Order type must be the same as the list type

### remove, unlist, relist

In [ ]:
random_order = random.choice(orders)

print(random_order.id)
order_list.remove(random_order)
order_list[random_order.id]
# error

18


KeyError: 18

In [83]:
random_order = random.choice(orders)

print(random_order.id)
order_list.unlist(random_order, OrderStatus.FILLED)
print(order_list[random_order.id])
# inactive
order_list.relist(random_order)
print(order_list[random_order.id])
# active
order_list.expire(random_order)
print(order_list[random_order.id])
# expired
order_list.relist(random_order)
order_list.cancel(random_order)
print(order_list[random_order.id])
order_list.cancel(random_order)
# error order is already canceled


6
id=6 order_type=<OrderType.ASK: 'ask'> price=101.70131585168942 volume=32 owner_id=4 status=<OrderStatus.FILLED: 3> created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257) updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 970819) listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)
id=6 order_type=<OrderType.ASK: 'ask'> price=101.70131585168942 volume=32 owner_id=4 status=<OrderStatus.RESTORED: 6> created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257) updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 971817) listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 971817)
id=6 order_type=<OrderType.ASK: 'ask'> price=101.70131585168942 volume=32 owner_id=4 status=<OrderStatus.EXPIRED: 7> created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257) updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 971817) listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 971817)
id=6 order_type=<OrderType.ASK: 'ask'> price=101.70131585168942 volume=32 owner_id=4 status=<OrderStatus.CANCELLED: 5> 

ValueError: Order is not active.

In [84]:
order_list.relist(random_order)
order_list.unlist(random_order, OrderStatus.PARTIALLY_FILLED)
# error

ValueError: Order cannot be unlisted with active status.

### filling 

In [85]:
random_order = random.choice(orders)
print(random_order)
order_list.fill(random_order, 10)
print(order_list[random_order.id])

id=16 order_type=<OrderType.ASK: 'ask'> price=96.4238155325062 volume=30 owner_id=3 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 15, 188255) updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008) listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)
id=16 order_type=<OrderType.ASK: 'ask'> price=96.4238155325062 volume=20 owner_id=3 status=<OrderStatus.PARTIALLY_FILLED: 2> created=datetime.datetime(2024, 2, 24, 20, 20, 15, 188255) updated=datetime.datetime(2024, 2, 24, 20, 20, 16, 842710) listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)


In [86]:
order_list.fill(random_order, random_order.volume + 10)
# error

ValueError: Volume is greater than order volume

In [87]:
order_list.fill(random_order, -11)
# error

ValueError: Volume must be positive

In [88]:
random_order = random.choice(orders)
order_list.fill(random_order, random_order.volume)
# filled order will be unlisted
order_list[random_order.id]

Order(id=2, order_type=<OrderType.ASK: 'ask'>, price=100.66665333771964, volume=0, owner_id=10, status=<OrderStatus.FILLED: 3>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 17, 331215), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008))

In [89]:
order_list.bisect_left(100)

[Order(id=3, order_type=<OrderType.ASK: 'ask'>, price=100.02901457203792, volume=36, owner_id=6, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)),
 Order(id=18, order_type=<OrderType.ASK: 'ask'>, price=101.14021182702777, volume=44, owner_id=2, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 188255), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)),
 Order(id=8, order_type=<OrderType.ASK: 'ask'>, price=101.65510958678865, volume=52, owner_id=2, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)),
 Order(id=6, order_type=<OrderType.ASK: 'ask'>, price=101.70131585168

### modify tests

In [90]:
print(len(order_list))
order_list.clear()
order_list

19


SortedList([])

In [91]:
orders = []
order_list = OrderList(order_type=OrderType.BID)
for i in range(3):
    orders.append(
        Order(
            id=i+1,
            order_type=OrderType.BID,
            price=random.uniform(95, 110),
            volume=random.randint(10, 70),
            owner_id=random.randint(1, 10),
        )
    )
for order in orders:
    order_list.add(order)
print(len(order_list))

for order in order_list:
    print(order)

3
id=1 order_type=<OrderType.BID: 'bid'> price=98.9423934225989 volume=47 owner_id=5 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)
id=3 order_type=<OrderType.BID: 'bid'> price=104.28419556843376 volume=18 owner_id=10 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)
id=2 order_type=<OrderType.BID: 'bid'> price=107.0168944192405 volume=65 owner_id=9 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)


In [92]:
random_order = random.choice(orders)
print(random_order.id)
order_list.modify(random_order.id, price=100, volume=100)

for order in order_list:
    print(order)

1
id=1 order_type=<OrderType.BID: 'bid'> price=100 volume=100 owner_id=5 status=<OrderStatus.MODIFIED: 4> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 800212) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)
id=3 order_type=<OrderType.BID: 'bid'> price=104.28419556843376 volume=18 owner_id=10 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)
id=2 order_type=<OrderType.BID: 'bid'> price=107.0168944192405 volume=65 owner_id=9 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)


Order(id=1, order_type=<OrderType.BID: 'bid'>, price=100, volume=100, owner_id=5, status=<OrderStatus.MODIFIED: 4>, created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659), updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 800212), listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659))